In [1]:
import pandas as pd

In [2]:
from keras.preprocessing import sequence
from keras.layers import Embedding, Dense, Flatten
from keras.models import Sequential


In [3]:
from sklearn import preprocessing


In [11]:
df_corpus = pd.read_csv('../output/word2vec_corpus.csv', index_col=0)
df_corpus.head(5)

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
Frase,,,,,,,,,,,,,,,,,,,,,
"['minuto', 'touro', 'ouro']",0.078066,-0.681861,-0.242816,-0.287935,-0.052421,0.331101,-0.139809,-0.379450,0.267952,0.175391,...,-0.211810,0.132065,-0.106570,0.100519,0.056235,0.211425,0.015619,-0.055449,-0.076335,1
['sim'],0.219127,-0.408984,-0.176726,-0.270843,0.154609,0.169897,-0.016856,-0.248304,0.197370,0.042259,...,-0.338304,0.263537,0.065959,0.021564,0.175807,0.181400,-0.136190,-0.056663,-0.056629,0
"['queria', 'saber', 'banking', 'própria']",0.139843,-0.536218,-0.217435,-0.219593,-0.017553,0.193954,-0.051410,-0.258538,0.166662,0.045931,...,-0.251460,0.155440,-0.043917,0.123936,0.185161,0.097498,-0.046922,-0.040375,-0.083413,1
"['palmas', 'palmas', 'palmas', 'palmas']",0.146434,-0.234622,-0.173836,-0.255587,0.160331,0.161181,-0.023813,-0.191149,0.076497,0.059744,...,-0.356565,0.168509,0.098124,0.088115,0.154685,0.217300,-0.187343,-0.033554,-0.157490,1
"['vamos', 'ajudar', 'brasil', 'principal', 'categoria', 'automobilismo']",0.151519,-0.500928,-0.217571,-0.271197,-0.014306,0.233144,-0.057781,-0.306150,0.215858,0.078090,...,-0.231650,0.143755,-0.044758,0.106567,0.123410,0.163982,-0.049926,-0.079451,-0.083599,1


In [10]:
df_cbow = pd.read_csv('../output/word2vec_cbow.csv', index_col=0)
df_cbow.head(5)

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
Frase,,,,,,,,,,,,,,,,,,,,,
"['minuto', 'touro', 'ouro']",0.194111,-0.086163,0.150576,-0.034733,-0.066786,0.122470,0.027893,0.064956,0.005728,0.135925,...,0.046650,-0.142142,0.131115,0.063196,0.025213,0.094671,-0.001749,-0.002428,0.263365,1
['sim'],0.109873,-0.062076,0.187268,-0.213734,0.046350,0.118452,-0.025577,-0.149704,0.143837,0.184814,...,0.031963,-0.051879,0.235329,-0.005905,0.000182,-0.029570,-0.064395,-0.025853,0.185357,0
"['queria', 'saber', 'banking']",0.114653,-0.154823,0.213298,-0.070127,0.017422,0.120094,0.008558,0.016445,0.034044,0.081284,...,0.023741,-0.061997,0.112366,0.059400,-0.006027,0.117726,-0.055369,-0.011474,0.152972,1
"['palmas', 'palmas', 'palmas', 'palmas']",0.215186,-0.092701,0.254579,-0.083714,0.099877,0.371233,0.050387,-0.127568,0.118620,0.129699,...,-0.090724,0.021903,0.221911,0.072890,0.119573,-0.004307,-0.091760,0.027266,0.144531,1
"['vamos', 'ajudar', 'brasil', 'principal', 'categoria', 'automobilismo', 'do']",0.204283,-0.116142,0.207498,-0.040869,0.002747,0.203083,0.013339,0.027435,0.052543,0.107897,...,-0.011755,-0.058785,0.153089,0.070365,-0.001163,0.095111,-0.030487,0.005549,0.144212,1


In [8]:
from sklearn.model_selection import train_test_split


In [9]:
target = df_cbow['sentimento']
features = df_cbow.iloc[:,0:50]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)


In [19]:
vocab_size = len(target) + 1


In [20]:
embeddings = Embedding(vocab_size, 100, weights=23, input_length=4, trainable=False)


In [21]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense
# Neural network
model = Sequential()
model.add(embeddings)
model.add(Dense(70, input_dim=50, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(30, activation='relu'))
model.add(Dense(1, activation='softmax'))


TypeError: object of type 'int' has no len()

In [12]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:
history = model.fit(X_train, y_train, epochs=100, batch_size=64)


Epoch 1/100
71/71 [==============================] - 2s 3ms/step - loss: 0.3189 - accuracy: 0.3854
Epoch 2/100
71/71 [==============================] - 0s 3ms/step - loss: 0.1287 - accuracy: 0.3854
Epoch 3/100
71/71 [==============================] - 0s 3ms/step - loss: -0.2162 - accuracy: 0.3854
Epoch 4/100
71/71 [==============================] - 0s 3ms/step - loss: -2.0368 - accuracy: 0.3854
Epoch 5/100
71/71 [==============================] - 0s 2ms/step - loss: -10.4616 - accuracy: 0.3854
Epoch 6/100
71/71 [==============================] - 0s 3ms/step - loss: -36.5121 - accuracy: 0.3854
Epoch 7/100
71/71 [==============================] - 0s 2ms/step - loss: -95.7558 - accuracy: 0.3854
Epoch 8/100
71/71 [==============================] - 0s 3ms/step - loss: -200.2197 - accuracy: 0.3854
Epoch 9/100
71/71 [==============================] - 0s 3ms/step - loss: -373.0764 - accuracy: 0.3854
Epoch 10/100
71/71 [==============================] - 0s 2ms/step - loss: -636.2449 - accuracy:

Epoch 79/100
71/71 [==============================] - 0s 3ms/step - loss: -1543663.5000 - accuracy: 0.3854
Epoch 80/100
71/71 [==============================] - 0s 3ms/step - loss: -1607483.2500 - accuracy: 0.3854
Epoch 81/100
71/71 [==============================] - 0s 3ms/step - loss: -1673091.1250 - accuracy: 0.3854
Epoch 82/100
71/71 [==============================] - 0s 3ms/step - loss: -1739423.1250 - accuracy: 0.3854
Epoch 83/100
71/71 [==============================] - 0s 3ms/step - loss: -1809267.6250 - accuracy: 0.3854
Epoch 84/100
71/71 [==============================] - 0s 3ms/step - loss: -1879099.1250 - accuracy: 0.3854
Epoch 85/100
71/71 [==============================] - 0s 3ms/step - loss: -1951391.1250 - accuracy: 0.3854
Epoch 86/100
71/71 [==============================] - 0s 3ms/step - loss: -2024756.7500 - accuracy: 0.3854
Epoch 87/100
71/71 [==============================] - 0s 3ms/step - loss: -2101302.2500 - accuracy: 0.3854
Epoch 88/100
71/71 [=================

In [15]:
y_pred = model.predict(X_test)


36/36 [==============================] - 0s 1ms/step
